In [ ]:
voevent_json=open('data/test/voevent-spiacs.json').read() #http://www.w3.org/2001/XMLSchema#str

In [ ]:
import oda

parsed_data = oda.evaluate("odahub","eventparser","ParseVOEvent",
                    voevent_json=voevent_json,
                  )

print("parsed data:", parsed_data)

In [ ]:
lvc_data = oda.evaluate("odahub","eventparser","ParseVOELVC",
                    voevent_json=voevent_json,
                    _async_return=True,
                  )

try:
    gwFAR = lvc_data['lvc_notice_data']['What']['Param']['FAR']
except:
    gwFAR = None
    
if lvc_data == "incomplete" or lvc_data is None:
    lvc_data = {}

print("lvc_data",lvc_data)

In [ ]:
#gw_FAR = lvc_data['lvc_notice_data']['What']['Param']

try:
    useful_data = lvc_data['lvc_notice_data']['What']['Group']['unnamed']
except:
    useful_data = dict(HasNS=0)

print("useful data:",useful_data)

In [ ]:

try:
    c=oda.evaluate("odahub", "integral-visibility", "integral-visibility",
            tstart_utc=parsed_data['utc_isot'],
            target_loc="healpix",
            target_healpix_url=parsed_data['lvc_data']['localization_url'],
            _async_return=True,
        )
except:
    c=oda.evaluate("odahub", "integral-visibility", "integral-visibility",
            tstart_utc=parsed_data['utc_isot'],
            target_loc="point",
            target_ra=parsed_data['ra_deg'],
            target_dec=parsed_data['dec_deg'],
            _async_return=True,
        )
    
print("visibility returns",c)

visibility_data = dict([(k,v) for k,v in c.items() if 'content' not in k])

visibility_data



In [ ]:
try:
    import integralclient as ic 

    sc=ic.get_sc(utc=parsed_data['utc_isot'],ra=parsed_data['ra_deg'],dec=parsed_data['dec_deg'])
except:
    sc={}
    
sc

In [ ]:
from astropy.time import Time, TimezoneInfo
from astropy import units as u
t=Time(parsed_data['utc_isot'],format='isot')
td=t.to_datetime(timezone=TimezoneInfo(utc_offset=2*u.hour))


In [ ]:
import numpy as np


advices = []

importance_levels=[
        'very high',
        'high',
        'medium',
        'low',
    ]
advice2level = lambda advice:importance_levels.index(advice['importance'])


if useful_data.get('HasNS', 0) == 1:
    advices.append(dict(importance='very high', reason = 'NS')) 

if visibility_data['visibility_summary']['esac']['visible'] > 40:
    advices.append(dict(importance='medium', reason = 'visible')) 
    
if lvc_data.get('loc_area_90_deg2',0)<300:
    advices.append(dict(importance='high', reason = 'small region')) 

if sc.get('theta',180)<15:
    advices.append(dict(importance='high', reason = 'in FoV (%.3lg deg)'%sc.get('theta',180))) 

if sc.get('theta',180)<40:
    advices.append(dict(importance='high', reason = 'close to FoV (%.3lg deg)'%sc.get('theta',180))) 

distance = visibility_data.get('extract',{}).get('DISTMEAN',None)
if distance is not None and distance < 500:
    advices.append(dict(importance='very high', reason = 'nearby: %.5lg Mpc'% visibility_data['extract']['DISTMEAN']))

signalness = -np.log10(gwFAR) if gwFAR is not None else None
if signalness is not None and signalness > 50:    
    advices.append(dict(importance='very high', reason = 'very rare: FAR %.5lg'% gwFAR))
    
if td.hour>8 and td.hour<17:
    advices.append(dict(importance='high', reason = 'working hours')) 
    

if (td.hour>8 and td.hour<17) and \
   (distance is not None and distance < 500) and \
   (sc.get('theta',180)<40) and \
   (visibility_data['visibility_summary']['esac']['visible'] > 40):
    advices.append(dict(importance='follow-up advised', reason = 'time, distance, separation, visibility aligned'))
    
advices

In [ ]:
# other avail obs
# consider other instrumnent detections
# consider reach in distance: RPOS or FTOO, cmplex slew
# complex cases
# disconnected regions
# check duty cycle, when data ready if
# property of current obs

In [ ]:
followup_grade_i = min(map(advice2level, advices))
followup_grade = importance_levels[followup_grade_i]

comments = "; ".join([advice['reason']+" ("+advice['importance']+")" for advice in sorted(advices, key=advice2level)])
comments

In [ ]:
if followup_grade_i == 0:
    recommended_action=dict(
            route = "SMS",
            destination = ["shift", "core"],
            comment = comments
    )
else:
    recommended_action=dict(
            route = "email",
            destination = ["shift", "core"],
            comment = comments
    )

In [ ]:
followup_grade=followup_grade
advices=advices
hasNS=useful_data['HasNS']
recommended_action=recommended_action
distance=distance
visibility_data=visibility_data